In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
df = pd.read_csv("cleaned_data.csv")
df = df.sample(frac=1)
df['gender'].replace(["Male", "Female"], [0, 1], inplace=True)
df['disposition'].replace(["Discharge", "Admit"], [0,  1], inplace=True)
df['esi'] = df['esi'] - 1
df['age'] = df['age'] / 100

In [ ]:
cc_cols = [x for x in df.columns if x[:3] == "cc_"]
med_cols = [x for x in df.columns if x[:5] == "meds_"]
with open("pmh_cols.txt") as f:
    pmh_cols = f.readlines()
pmh_cols = list(map(lambda x: x[:-1], pmh_cols))

x_cols = ["age", "gender"] + cc_cols + pmh_cols
y_cols = ["disposition"] + med_cols

In [ ]:
val_split = 0.9

np_x_train = np.array(df[x_cols])[:int(val_split * df.shape[0]), :]
np_y_train = np.array(df[y_cols])[:int(val_split * df.shape[0]), :]

np_x_val = np.array(df[x_cols])[int(val_split * df.shape[0]):, :]
np_y_val = np.array(df[y_cols])[int(val_split * df.shape[0]):, :]

In [ ]:
gen_model = tf.keras.Sequential([
    tf.keras.layers.Input(483),
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(49, activation="sigmoid")
])

smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.001, decay_steps=15, decay_rate=0.96, staircase=True
)

gen_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=dice_coef_loss,
    metrics=[
        dice_coef,
        tf.keras.metrics.AUC(curve="PR"),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall()
    ]
)

In [ ]:
gen_model.fit(
    x=np_x_train,
    y=np_y_train,
    epochs=30,
    validation_data=(np_x_val, np_y_val),
    batch_size=32,
    callbacks=[tf.keras.callbacks.ModelCheckpoint(
        "./model_best.h5",
        save_weights_only=True,
        save_best_only=True
    )]
)